## 오류 해결 방법을 못찾아 소스코드만 업로드
  - 하루 동안 오류를 해결하려고 했지만 결국은 성공하지 못했습니다
  - 코드 전체를 다시 읽고 수정할려 하였지만 해결책을 찾을 수 없었습니다
  - 오류부분 코딩과 오류메시지를 앞에 올려봅니다
  - 과제 제출 시간이라 일단 제출하고, 오류해결 방법은 계속 찾아볼 예정입니다.


'''python

        EPOCHS = 10

        for epoch in range(EPOCHS):
            total_loss = 0

            idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
            random.shuffle(idx_list)
            t = tqdm(idx_list)

            for (batch, idx) in enumerate(t):
                batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                                       dec_train[idx:idx+BATCH_SIZE],
                                       encoder,
                                       decoder,
                                       optimizer,
                                       dec_tokenizer)
                total_loss += batch_loss

                t.set_description_str('Epoch %2d' %(epoch + 1))
                t.set_postfix_str('Loss %.4f' %(total_loss.numpy() / (batch+1)))
'''

'''python  

          0%|          | 0/694 [01:19<?, ?it/s]

    ---------------------------------------------------------------------------
    ResourceExhaustedError                    Traceback (most recent call last)
    <ipython-input-23-69b7617f2f0a> in <module>
         17                                decoder,
         18                                optimizer,
    ---> 19                                dec_tokenizer)
         20         total_loss += batch_loss
         21 

    ~/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py in __call__(self, *args, **kwds)
        578         xla_context.Exit()
        579     else:
    --> 580       result = self._call(*args, **kwds)
        581 
        582     if tracing_count == self._get_tracing_count():

    ~/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py in _call(self, *args, **kwds)
        642         # Lifting succeeded, so variables are initialized and we can run the
        643         # stateless function.
    --> 644         return self._stateless_fn(*args, **kwds)
        645     else:
        646       canon_args, canon_kwds = \

    ~/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/eager/function.py in __call__(self, *args, **kwargs)
       2418     with self._lock:
       2419       graph_function, args, kwargs = self._maybe_define_function(args, kwargs)
    -> 2420     return graph_function._filtered_call(args, kwargs)  # pylint: disable=protected-access
       2421 
       2422   @property

    ~/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/eager/function.py in _filtered_call(self, args, kwargs)
       1663          if isinstance(t, (ops.Tensor,
       1664                            resource_variable_ops.BaseResourceVariable))),
    -> 1665         self.captured_inputs)
       1666 
       1667   def _call_flat(self, args, captured_inputs, cancellation_manager=None):

    ~/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/eager/function.py in _call_flat(self, args, captured_inputs, cancellation_manager)
       1744       # No tape is watching; skip to running the function.
       1745       return self._build_call_outputs(self._inference_function.call(
    -> 1746           ctx, args, cancellation_manager=cancellation_manager))
       1747     forward_backward = self._select_forward_and_backward_functions(
       1748         args,

    ~/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/eager/function.py in call(self, ctx, args, cancellation_manager)
        596               inputs=args,
        597               attrs=attrs,
    --> 598               ctx=ctx)
        599         else:
        600           outputs = execute.execute_with_cancellation(

    ~/anaconda3/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/eager/execute.py in quick_execute(op_name, num_outputs, inputs, attrs, ctx, name)
         58     ctx.ensure_initialized()
         59     tensors = pywrap_tfe.TFE_Py_Execute(ctx._handle, device_name, op_name,
    ---> 60                                         inputs, attrs, num_outputs)
         61   except core._NotOkStatusException as e:
         62     if name is not None:

    ResourceExhaustedError:  [_Derived_]  OOM when allocating tensor with shape[512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
         [[{{node transpose_4}}]]
    Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

         [[decoder_64/gru_1/StatefulPartitionedCall]]
    Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

         [[concat/_792]]
    Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
     [Op:__inference_train_step_143096]

    Function call stack:
    train_step -> train_step -> train_step

'''

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truethype/nanum/NanumBarunGothic.ttf'

font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager._rebuild()

In [3]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

In [4]:
f_kor = open("/home/aiffel/work/goinDeeper/goDeeper/go_08 seq2seq/data/korean-english-park.train.ko", 'r')
f_eng = open("/home/aiffel/work/goinDeeper/goDeeper/go_08 seq2seq/data/korean-english-park.train.en", 'r')

lines_kor = f_kor.readlines()
lines_eng = f_eng.readlines()

f_kor.close()
f_eng.close()

len(lines_kor), len(lines_eng)

(94123, 94123)

In [5]:
lines_kor[:5], lines_eng[:5]

(['개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"\n',
  '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.\n',
  '그러나 이것은 또한 책상도 필요로 하지 않는다.\n',
  '79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다.\n',
  '정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔음을 밝혔으며, 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다.\n'],
 ['Much of personal computing is about "can you top this?"\n',
  'so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.\n',
  "Like all optical mice, But it also doesn't need a desk.\n",
  'uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.\n',
  "Intelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror attack.\n"])

In [6]:
# 각 문장의 끝에 있는 개행문자 삭제

search = '\n'
lines_kor = [word.strip(search) for word in lines_kor]
lines_eng = [word.strip(search) for word in lines_eng]

In [7]:
lines_kor[:5], lines_eng[:5]

(['개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"',
  '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.',
  '그러나 이것은 또한 책상도 필요로 하지 않는다.',
  '79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다.',
  '정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔음을 밝혔으며, 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다.'],
 ['Much of personal computing is about "can you top this?"',
  'so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.',
  "Like all optical mice, But it also doesn't need a desk.",
  'uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.',
  "Intelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror attack."])

In [8]:
# 노드에 나온 형태로 만들었지만 의미 없음
#   --> 다시 '\t'에 따라 분리해야 하므로

# 중복 제거할 때 이용할 수 있음. 리스트를 합친 것을 제거하기에 영문과 같이 생각하면 됨.

raw = []
for i in range(len(lines_kor)):
    st = lines_kor[i] + '\t' + lines_eng[i]
    raw.append(st)
    
raw[:5]

['개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"\tMuch of personal computing is about "can you top this?"',
 '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.\tso a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.',
 "그러나 이것은 또한 책상도 필요로 하지 않는다.\tLike all optical mice, But it also doesn't need a desk.",
 '79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다.\tuses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.',
 "정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔음을 밝혔으며, 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다.\tIntelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror attack."]

In [9]:
print("Data Size: ", len(raw))
print("Example: ")

for sen in raw[0:100][::20]:
    print(">>", sen)

Data Size:  94123
Example: 
>> 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"	Much of personal computing is about "can you top this?"
>> 북한의 핵무기 계획을 포기하도록 하려는 압력이 거세지고 있는 가운데, 일본과 북한의 외교관들이 외교 관계를 정상화하려는 회담을 재개했다.	Amid mounting pressure on North Korea to abandon its nuclear weapons program Japanese and North Korean diplomats have resumed talks on normalizing diplomatic relations.
>> "경호 로보트가 침입자나 화재를 탐지하기 위해서 개인적으로, 그리고 전문적으로 사용되고 있습니다."	“Guard robots are used privately and professionally to detect intruders or fire,” Karlsson said.
>> 수자원부 당국은 논란이 되고 있고, 막대한 비용이 드는 이 사업에 대해 내년에 건설을 시작할 계획이다.	Authorities from the Water Resources Ministry plan to begin construction next year on the controversial and hugely expensive project.
>> 또한 근력 운동은 활발하게 걷는 것이나 최소한 20분 동안 뛰는 것과 같은 유산소 활동에서 얻는 운동 효과를 심장과 폐에 주지 않기 때문에, 연구학자들은 근력 운동이 심장에 큰 영향을 미치는지 여부에 대해 논쟁을 해왔다.	Researchers also have debated whether weight-training has a big impact on the heart, since it does not give the heart and lungs the kind of workout t

In [10]:
clean_corpus = list(set(raw))
len(clean_corpus)

78968

In [11]:
from konlpy.tag import Mecab

def preprocess_sentence(sentence, s_token=False, e_token=False):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    if s_token:
        sentence = re.sub(r"[^a-zA-Z?.!,0-9]+", " ", sentence)
        sentence = sentence.strip()
        sentence = '<start> ' + sentence
        sentence += ' <end>'
        sentence = sentence.split()
            
    else:
        mecab = Mecab()
        sentence = re.sub(r"[^ㄱ-하-ㅣ가-힣a-zA-Z 0-9 ]", " ", sentence)
        sentence = mecab.morphs(sentence)

    return sentence

In [12]:
enc_corpus = []
dec_corpus = []

# num_examples = 20000

for pair in clean_corpus:
    kor, eng = pair.split("\t")
    
    if len(kor) <= 80:
        enc_corpus.append(preprocess_sentence(kor))
        dec_corpus.append(preprocess_sentence(eng, 
                            s_token=True, e_token=True))
    
print("Korean: ", enc_corpus[80], "  len(encoder): ", len(enc_corpus))
print("English: ", dec_corpus[80], "  len(decoder): ", len(dec_corpus))

Korean:  ['이', '수치', '는', '통계청', '이', '2003', '년', '데이터', '를', '집계', '하', '기', '시작', '한', '이래', '최고', '치이', '다']   len(encoder):  59143
English:  ['<start>', 'The', 'figure', 'was', 'the', 'highest', 'since', 'the', 'office', 'began', 'compiling', 'data', 'in', '2003', '.', '<end>']   len(decoder):  59143


In [13]:
print("Korean: ", enc_corpus[820])
print("English: ", dec_corpus[820])

Korean:  ['크라이튼', '은', 'TV', '연속극', 'ER', '로', '에', '이미', '상', '방송', '저널리즘', '의', '권위', '있', '는', '피바디', '상', '미국', '작가', '조합', '상', '등', '여러', '시상식', '에서', '상', '을', '받', '았', '다']
English:  ['<start>', 'Crichton', 'won', 'an', 'Emmy', ',', 'a', 'Peabody', ',', 'a', 'Writers', 'Guild', 'of', 'America', 'Award', 'for', 'ER', ',', 'and', 'won', 'other', 'awards', 'as', 'well', '.', '<end>']


In [14]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(
        tensor, padding='post')
    
    return tensor, tokenizer

In [15]:
enc_tensor, enc_tokenizer = tokenize(enc_corpus)
dec_tensor, dec_tokenizer = tokenize(dec_corpus)

enc_train, enc_val, dec_train, dec_val = \
    train_test_split(enc_tensor, dec_tensor, test_size=0.25)

print("Korean Vocab Size: ", len(enc_tokenizer.index_word))
print("English Vocab Size: ", len(dec_tokenizer.index_word))

enc_tensor.shape

Korean Vocab Size:  41860
English Vocab Size:  38781


(59143, 45)

In [16]:
del enc_corpus
del dec_corpus
del clean_corpus

In [17]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
        
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]
        
        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)
        
        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)
        
        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)
        
        return context_vec, attn

In [18]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        # TODO: Awesome Encoder Modules
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units, return_sequences=True)
        
    def call(self, x):
        # TODO: Awesome Process
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

In [19]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        # TODO: Awesome Decoder Modules
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units, return_sequences=True,
                                      return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        self.attention = BahdanauAttention(self.dec_units)
        
    def call(self, x, h_dec, enc_out):
        # TODO: Awesome Process
        context_vec, attn = self.attention(enc_out, h_dec)
        
        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)
        
        return out, h_dec, attn

In [20]:
BATCH_SIZE = 64
SRC_VOCAB_SIZE = len(enc_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer.index_word) + 1

units = 512
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print('Encoder Output: ', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(
    tf.random.uniform((BATCH_SIZE, 1)), sample_state, sample_output)

print('Decoder Output: ', sample_logits.shape)
print('Decoder Hidden State: ', h_dec.shape)
print('Attention: ', attn.shape)

Encoder Output:  (64, 30, 512)
Decoder Output:  (64, 38782)
Decoder Hidden State:  (64, 512)
Attention:  (64, 30, 1)


In [21]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [22]:
@tf.function

def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([
            dec_tok.word_index['<start>']] * bsz, 1)
        
        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)
            
            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
            
    batch_loss = (loss / int(tgt.shape[1]))
    
    variables = encoder.trainable_variables + \
                decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

In [23]:
from tqdm import tqdm
import random

EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)
    
    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                               dec_train[idx:idx+BATCH_SIZE],
                               encoder,
                               decoder,
                               optimizer,
                               dec_tokenizer)
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' %(epoch + 1))
        t.set_postfix_str('Loss %.4f' %(total_loss.numpy() / (batch+1)))

  0%|          | 0/694 [01:19<?, ?it/s]


ResourceExhaustedError:  [_Derived_]  OOM when allocating tensor with shape[512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node transpose_4}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[decoder_64/gru_1/StatefulPartitionedCall]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[concat/_792]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_step_143096]

Function call stack:
train_step -> train_step -> train_step


In [62]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_train.shape[-1], enc_train.shape[-1]))
    
    sentence = preprocess_sentence(sentence)
    
    inputs = enc_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(
        inputs,
        maxlen=enc_train.shape[-1],
        padding='post')
    
    result = ''
    
    enc_out = encoder(inputs)
    
    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([
        dec_tokenizer.word_index['<start>']], 0)
    
    for t in range(dec_train.shape[-1]):
        prediction, dec_hidden, attention_weights = \
            decoder(dec_input, dec_hidden, enc_out)
        
        attention_weights = tf.reshape(attention_weights, (-1,))
        attention[t] = attention_weights.numpy()
        
        predicted_id = \
            tf.argmax(tf.math.softmax(prediction, axis=-1)[0]).numpy()
        
        result += dec_tokenizer.index_word[predicted_id] + ' '
        
        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention
        
        dec_input = tf.expand_dims([predicted_id], 0)
        
    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    plt.show()
    
    
def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)
    
    print('Input: %s' %(sentence))
    print('Predicted translatio: {}'.format(result))
    
    attention = \
        attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))

In [1]:
translate("오바마는 대통령이다.", encoder, decoder)
translate("시민들은 도시 속에 산다.", encoder, decoder)
translate("커피는 필요 없다.", encoder, decoder)
translate("일곱 명의 사망자가 발생했다.", encoder, decoder)